In [1]:
!pip install evaluate==0.4.3
!pip install llama-cpp-python==0.1.9
!pip install pinecone-client==5.0.1
!pip install langchain_community==0.2.16
!pip install langchain-chroma==0.1.4
!pip install chromadb==0.5.11
!pip install sentence-transformers==3.1.1
!pip install ctransformers

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: omegaconf 2.0.6 has a non-sta

In [2]:
from langchain_community.document_loaders import PDFMinerLoader, TextLoader, CSVLoader, UnstructuredWordDocumentLoader, UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from multiprocessing.pool import ThreadPool
from langchain_chroma.vectorstores import Chroma
from langchain.schema import Document
from chromadb.config import Settings
from chromadb import Client
from llama_cpp import Llama
from evaluate import load
from typing import Any
from tqdm import tqdm
from pathlib import Path

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import numpy as np
import statistics
import pinecone
import glob
import os

#Лабораторная работа №5

##Declaring constant

In [4]:
path_to_index = '/content/VDB'
path_to_documents = 'nlp-24-autumn/projects/dataset/20news-bydate-train/comp.graphics' #49960.txt

In [5]:
# Словарь, сопоставляющий расширения файлов с соответствующими загрузчиками данных и их параметрами
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".txt": (TextLoader, {"encoding": "ISO-8859-1"}),
}

In [6]:
# Параметры конфигурации для векторного поиска и разделения текста
INDEX_NAME = "VDB"  # Название индекса для хранения векторных представлений
COLLECTION_NAME = "document_collection"
EMBEDDINGS = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"  # Название модели эмбеддингов, используемой для векторизации текстов
SIZE = 250  # Размер фрагмента текста для разделения документов
OVERLAP = 50  # Перекрытие между фрагментами текста для обеспечения контекста

## Loader

In [7]:
# Класс для загрузки документов из различных источников, поддерживающий работу с разными форматами файлов
class Loader:
    def load_single_document(self, file_path: str):
        # Метод для загрузки одного документа на основе пути к файлу
        ext = Path(file_path).suffix.lower()
        if ext in LOADER_MAPPING:
            loader_class, loader_args = LOADER_MAPPING[ext]
            loader_args['file_path'] = file_path;
            loader = loader_class(**loader_args)
            document = loader.load()
            return document
        else:
            raise ValueError(f"Unsupported file extension: {ext}")

    def load_documents(self, source_dir: str):
        # Метод для загрузки всех документов из указанной директории
        documents = []
        if os.path.isfile(source_dir):
          documents.extend(self.load_single_document(file_path))
        else:
          for root, _, files in os.walk(source_dir):
              for file_name in files:
                  file_path = os.path.join(root, file_name)
                  try:
                      document = self.load_single_document(file_path)
                      documents.extend(document)
                  except ValueError as e:
                      print(e)
        return documents

In [8]:
loader = Loader()

example_document = loader.load_documents(path_to_documents)

example_document

[]

## Splitter

In [9]:
# Класс для разделения документов на фрагменты определённого размера с заданным перекрытием
class Splitter:
    def __init__(self, chunk_size, chunk_overlap):
        # Инициализация параметров разделения: размер фрагмента и величина перекрытия
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_documents(self, documents):
        fragments = []
        for document in documents:
            text = document.page_content
            doc_meta = document.metadata
            start = 0

            while start < len(text):
                end = min(start + self.chunk_size, len(text))
                fragment_text = text[start:end]

                fragment = {
                    "text": fragment_text,
                    "metadata": doc_meta
                }

                fragments.append(fragment)
                start += self.chunk_size - self.chunk_overlap

        return fragments


In [10]:
splitter = Splitter(SIZE, OVERLAP)

example_fragments = splitter.split_documents(example_document)

example_fragments

[]

##Vector database

In [11]:
from sentence_transformers import SentenceTransformer

# Базовый класс для создания эмбеддингов, обеспечивающий интерфейс для получения модели эмбеддингов
class Embedder:
    def __init__(self, model_name: str):
        # Инициализация эмбеддера
        self.model = SentenceTransformer(model_name)

    def get_embedder(self):
        # Метод для получения модели эмбеддингов, которая будет использоваться для векторизации текстов
        return self.model

    def encode(self, texts: list[dict]):
        return self.model.encode(texts)

In [12]:
class HuggingFaceEmbedder(Embedder):
    def __init__(self):
        super().__init__(EMBEDDINGS)

In [13]:
embedder = HuggingFaceEmbedder()

example_embedded_fragments = embedder.encode(example_fragments)

example_embedded_fragments

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([], dtype=float64)

### Базовые классы

In [14]:
class Element():
  def __init__(self, embedding, metadata):
    self.embedding = embedding
    self.metadata = metadata

# Базовый класс для работы с коллекцией документов, поддерживающий добавление, поиск и очистку данных
class Collector:
    def __init__(self, splitter: Splitter, embedder: Embedder):
        self.splitter = splitter
        self.embedder = embedder
        self.loader = Loader()
        self.documents = []

    def add(self, texts: list[str], metadatas: list[dict]):
        # Метод для добавления текстов и связанных с ними метаданных в коллекцию
        added_documents = []
        embeddings = self.embedder.encode(texts)
        for embedding, metadata in zip(embeddings, metadatas):
            self.documents.append(Element(embedding=embedding, metadata=metadata))
            added_documents.append(Element(embedding=embedding, metadata=metadata))
        return added_documents

    def add_from_directory(self, dir_path: str):
        # Метод для добавления документов в коллекцию из указанной директории
        documents = self.loader.load_documents(dir_path)
        fragments = self.splitter.split_documents(documents)

        texts = [fragment["text"] for fragment in fragments]
        metadatas = [{"file_path": fragment["metadata"]} for fragment in fragments]

        self.add(texts, metadatas)

    def query_documents(self, embedding, top_k):
        pass

    def _cosine_similarity(self, vec1, vec2):
        dot_product = sum(a * b for a, b in zip(vec1, vec2))
        norm1 = sum(a ** 2 for a in vec1) ** 0.5
        norm2 = sum(b ** 2 for b in vec2) ** 0.5
        return dot_product / (norm1 * norm2)

    def get(self, search_strings: list[str], n_results: int) -> list[Document]:
        # Метод для поиска документов по строкам запроса с ограничением на количество результатов
        search_embeddings = self.embedder.encode(search_strings)
        results = []
        for search_embedding in search_embeddings:
            result_docs = self.query_documents(search_embedding, top_k=n_results)
            results.extend(result_docs)
        return results

    def get_documents(self, search_string: str, n_results: int, score_threshold: float) -> list[Document]:
        # Метод для поиска документов с учётом порога релевантности и количества возвращаемых результатов
        search_embedding = self.embedder.encode([search_string])[0]
        result_docs = self.query_documents(search_embedding, top_k=n_results)
        return [doc for doc in result_docs if doc.score >= score_threshold]

    def clear(self):
        # Метод для очистки коллекции документов
        self.documents.clear()

In [15]:
class ChromaCollector(Collector):
    def __init__(self, splitter: Splitter, embedder: HuggingFaceEmbedder):
        super().__init__(splitter, embedder)
        self.client = Client()
        self.collection = self.client.get_or_create_collection(COLLECTION_NAME)
        self.doc_id_counter = 0

    def add(self, texts: list[str], metadatas: list[dict]):
        added_documents = super().add(texts, metadatas)
        print(added_documents)

        embeddings = [doc.embedding.tolist() for doc in added_documents]
        ids = [f"id_{self.doc_id_counter + i}" for i in range(len(added_documents))]
        self.doc_id_counter += len(added_documents)

        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=[doc.metadata for doc in added_documents],
            ids=ids
        )

    # def query_documents(self, embedding, top_k):
    #     return self.collection.query(embedding, top_k=top_k)

    def query_documents(self, embedding, top_k):
        documents_with_scores = [(doc, self._cosine_similarity(embedding, doc.embedding)) for doc in self.collection]
        documents_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, score in documents_with_scores[:top_k]]

    def clear(self):
        super().clear()
        self.client.delete_collection(COLLECTION_NAME)

### Класс Collector

In [16]:
# Базовый класс для работы с коллекцией документов, поддерживающий добавление, поиск и очистку данных
class Collector:
    def __init__(self, splitter: Splitter, embedder: Embedder):
        self.splitter = splitter
        self.embedder = embedder
        self.loader = Loader()

    def add(self, texts: list[str], metadatas: list[dict]):
        # Метод для добавления текстов и связанных с ними метаданных в коллекцию
        embeddings = self.embedder.encode(texts)
        return [{"embedding": embedding, "metadata": metadata} for embedding, metadata in zip(embeddings, metadatas)]

    def add_from_directory(self, dir_path: str):
        # Метод для добавления документов в коллекцию из указанной директории
        documents = self.loader.load_documents(dir_path)
        fragments = self.splitter.split_documents(documents)

        texts = [fragment["text"] for fragment in fragments]
        metadatas = [fragment["metadata"] for fragment in fragments]
        self.add(texts, metadatas)

    def _cosine_similarity(self, vec1, vec2):
        dot_product = sum(a * b for a, b in zip(vec1, vec2))
        norm1 = sum(a ** 2 for a in vec1) ** 0.5
        norm2 = sum(b ** 2 for b in vec2) ** 0.5
        return dot_product / (norm1 * norm2)

    def query_documents(self, embedding, top_k):
        pass

    def get(self, search_strings: list[str], n_results: int) -> list[Document]:
        # Метод для поиска документов по строкам запроса с ограничением на количество результатов
        search_embeddings = self.embedder.encode(search_strings)
        results = []
        for search_embedding in search_embeddings:
            result_docs = self.query_documents(search_embedding, top_k=n_results)
            results.extend(result_docs)
        return results

    def get_documents(self, search_string: str, n_results: int, score_threshold: float) -> list[Document]:
        # Метод для поиска документов с учётом порога релевантности и количества возвращаемых результатов
        search_embedding = self.embedder.encode([search_string])[0]
        result_docs = self.query_documents(search_embedding, top_k=n_results)
        return [doc for doc in result_docs if doc.score >= score_threshold]


class ChromaCollector(Collector):
    def __init__(self, splitter: Splitter, embedder: HuggingFaceEmbedder):
        super().__init__(splitter, embedder)
        self.client = Client()
        self.collection = self.client.get_or_create_collection(COLLECTION_NAME)
        self.doc_id_counter = 0

    def add(self, texts: list[str], metadatas: list[dict]):
        # Добавляем документы с их эмбеддингами и метаданными в коллекцию
        added_documents = super().add(texts, metadatas)
        embeddings = np.array([doc["embedding"] for doc in added_documents]).astype("float32")

        ids = [f"id_{self.doc_id_counter + i}" for i in range(len(added_documents))]
        self.doc_id_counter += len(added_documents)

        self.collection.add(
            documents=texts,
            embeddings=embeddings.tolist(),
            metadatas=[doc["metadata"] for doc in added_documents],
            ids=ids
        )

    def query_documents(self, query_embedding, top_k: int):
        results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )

        ids = results['ids'][0]
        distances = results['distances'][0]
        metadatas = results['metadatas'][0]
        documents = results['documents'][0]

        results_list = []
        for i in range(len(ids)):
            results_list.append({
                "id": ids[i],
                "distance": distances[i],
                "metadata": metadatas[i],
                "document": documents[i]
            })

        return results_list


    def clear(self):
        self.client.delete_collection(COLLECTION_NAME)

###Implementation vector database

In [17]:
# path_to_index = '/content/VDB' #@param {type:"string"}
# path_to_documents = '/content/tmp' #@param {type:"string"}

In [18]:
#Нужно написать реализацию векторной базы данных

## Search

In [19]:
query = 'How can I clean a suede jacket?' #@param {type:"string"}
n_results = 5 #@param {type:"integer"}
score_threshold = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}


In [20]:
#Нужно реализовать эксперимент по поиску в векторном индексе
example_splitter = Splitter(SIZE, OVERLAP)
example_embedder = HuggingFaceEmbedder()
exampple_collector = ChromaCollector(splitter, embedder)

exampple_collector.clear()
exampple_collector = ChromaCollector(example_splitter, example_embedder)

exaple_documents = [
        "The sun is an average star.",
        "The moon orbits the Earth.",
        "I enjoy cooking and baking.",
        "The galaxy is vast and full of stars.",
        "Artificial intelligence is transforming industries."
    ]

example_metadatas = [
    {'file_path': 'dir_1'},
    {'file_path': 'dir_2'},
    {'file_path': 'dir_3'},
    {'file_path': 'dir_4'},
    {'file_path': 'dir_5'},
    ]


exampple_collector.add(exaple_documents, example_metadatas)
exampple_collector.collection.peek()

example_query = "What is sun?"
example_query_embedding = embedder.encode([example_query])[0]
exampple_collector.query_documents(example_query_embedding, 2)

[{'id': 'id_0',
  'distance': 8.91515064239502,
  'metadata': {'file_path': 'dir_1'},
  'document': 'The sun is an average star.'},
 {'id': 'id_3',
  'distance': 14.08548355102539,
  'metadata': {'file_path': 'dir_4'},
  'document': 'The galaxy is vast and full of stars.'}]

In [21]:
splitter = Splitter(SIZE, OVERLAP)
embedder = HuggingFaceEmbedder()
collector = ChromaCollector(splitter, embedder)

collector.clear()
collector = ChromaCollector(splitter, embedder)

In [22]:
path_to_documents = '../../dataset/20news-bydate-train/comp.graphics'

collector.add_from_directory(path_to_documents)

In [23]:
collector.collection.peek(500)

{'ids': ['id_0',
  'id_1',
  'id_2',
  'id_3',
  'id_4',
  'id_5',
  'id_6',
  'id_7',
  'id_8',
  'id_9',
  'id_10',
  'id_11',
  'id_12',
  'id_13',
  'id_14',
  'id_15',
  'id_16',
  'id_17',
  'id_18',
  'id_19',
  'id_20',
  'id_21',
  'id_22',
  'id_23',
  'id_24',
  'id_25',
  'id_26',
  'id_27',
  'id_28',
  'id_29',
  'id_30',
  'id_31',
  'id_32',
  'id_33',
  'id_34',
  'id_35',
  'id_36',
  'id_37',
  'id_38',
  'id_39',
  'id_40',
  'id_41',
  'id_42',
  'id_43',
  'id_44',
  'id_45',
  'id_46',
  'id_47',
  'id_48',
  'id_49',
  'id_50',
  'id_51',
  'id_52',
  'id_53',
  'id_54',
  'id_55',
  'id_56',
  'id_57',
  'id_58',
  'id_59',
  'id_60',
  'id_61',
  'id_62',
  'id_63',
  'id_64',
  'id_65',
  'id_66',
  'id_67',
  'id_68',
  'id_69',
  'id_70',
  'id_71',
  'id_72',
  'id_73',
  'id_74',
  'id_75',
  'id_76',
  'id_77',
  'id_78',
  'id_79',
  'id_80',
  'id_81',
  'id_82',
  'id_83',
  'id_84',
  'id_85',
  'id_86',
  'id_87',
  'id_88',
  'id_89',
  'id_90',
  

In [24]:
search_queries = ["What tools does include ImageMagick?"]

for query in search_queries:
    print(f"\nSearch query: {query}")
    results = collector.get([query], n_results=n_results)

    for i, doc in enumerate(results):
        print(f"Result {i + 1}: {doc}")


Search query: What tools does include ImageMagick?
Result 1: {'id': 'id_4483', 'distance': 4.114986419677734, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38403.txt'}, 'document': 'oftware for Amigas.  ImageMaster, from Black Belt Systems, is another\nwell-regarded commercial graphics package with JPEG support.\n\nThe free IJG JPEG software is available compiled for Amigas from\namiga.physik.unizh.ch (and mirror sites) in directory'}
Result 2: {'id': 'id_4396', 'distance': 4.434577941894531, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38403.txt'}, 'document': "isty's free ImageMagick package,\nalso available from export.lcs.mit.edu, file contrib/ImageMagick.tar.Z.\nThis package handles many image processing and conversion tasks.  The\nImageMagick viewer handles 24-bit displays correctly; for colormapped\ndispl"}
Result 3: {'id': 'id_3623', 'distance': 4.792247772216797, 'metadata': {'source': '../../dataset/20news-bydate-train/com

## Evaluation

In [25]:
# Класс для оценки работы коллектора, предоставляющий функционал для поиска, оценки и расчета статистики по результатам
class CollectorEvaluator:
    def __init__(self, collector: Collector, n_top=5):
        # Инициализация коллектора и параметра n_top для ограничения числа возвращаемых результатов
        self.collector = collector
        self.n_top = n_top

    def explore_collector(self, text: str):
        # Метод для поиска документов в коллекторе на основе текста запроса
        collector_results = self.collector.get([text], n_results=self.n_top)
        return collector_results

    def eval(self, query: str, answer: str):
        # Метод для оценки корректности найденных документов на основе запроса и правильного ответа
        collector_results = self.explore_collector(query)

        print(f"\nSearch query: {query},\nanswer: {answer}")

        for i, doc in enumerate(collector_results, start=1):
            print(f"Result {i}: {doc}")

        for i, doc in enumerate(collector_results, start=1):
            if answer in doc["document"]:
                return i
        return None

    def calculate_statistics(self, data: list[int]):
        # Метод для расчета статистических показателей (например, минимальное, максимальное, среднее значение)

        filtered_data = [serial_number for serial_number in data if serial_number is not None]
        not_found_queries = len([serial_number for serial_number in data if serial_number is None])

        if not filtered_data:
            return {"min": None, "max": None, "mean": None, "not found": None}

        min_serial_number = min(filtered_data)
        max_serial_number = max(filtered_data)
        mean_serial_number = sum(filtered_data) / len(filtered_data)
        return {"min": min_serial_number, "max": max_serial_number, "mean": mean_serial_number, "not_found": not_found_queries / len(data)}

    def explore_and_calculate(self, data: list[tuple[str, str]]):
        # Метод для проведения поиска по данным и расчета статистики на основе результатов
        collector_results = []
        for query, answer in data:
            serial_number = self.eval(query, answer)
            collector_results.append(serial_number)

        stats = self.calculate_statistics(collector_results)
        return stats

In [26]:
path_to_dataset = '/content/QA.csv' #@param {type:"string"}
n_lines = 100 #@param {type:"integer"}
n_top = 10 #@param {type:"integer"}



In [27]:
#Нужно написать эксперимент для оценки полученной коллекции
example_collector = ChromaCollector(example_splitter, example_embedder)
example_evaluator = CollectorEvaluator(example_collector, n_top=n_top)

example_data = [
    ("What is the sun?", "The sun is an average star."),
    ("What is the moon?", "The moon orbits the Earth."),
    ("Cooking tips", "I enjoy cooking and baking.")
]

example_stats = example_evaluator.explore_and_calculate(example_data)
print("Статистика по позициям релевантных ответов:", example_stats)



Search query: What is the sun?,
answer: The sun is an average star.
Result 1: {'id': 'id_2664', 'distance': 8.065162658691406, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38306.txt'}, 'document': '.Sun.COM\n'}
Result 2: {'id': 'id_4629', 'distance': 9.803832054138184, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38614.txt'}, 'document': 'ars\n'}
Result 3: {'id': 'id_4107', 'distance': 10.071667671203613, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38375.txt'}, 'document': 's on Sun 3/xxx, Sun 4/xxx (OS3.5, 4.0 and 4.0.3) under SunView.\n  The expert system for image segmentation is written in Allegro Common Lisp.\n  It was used on the following domains: computer science (image analysis), \n  medicine, biology, physics. It'}
Result 4: {'id': 'id_4096', 'distance': 10.396493911743164, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38375.txt'}, 'document': 'lled user, and programme

In [28]:
data = [
    ('What are some examples of toolkits that can be used for image format conversion and basic image manipulations?', 'umber of toolkits for converting from one image format to\nanother, doing simple image manipulations such as size scaling, plus\nthe above-mentioned 24 -> 8, color -> gray, gray -> b&w conversions.\nHere are pointers to some of them:\n\n    xv by John Bra'),
    ('What techniques are discussed in the context of quantizing 24-bit images down to 8 bits, and where can one find a relevant reference on this topic?', 'for\nshading, chapter 19 for clipping, and branch out from there.\n\n\n3) Quantizing 24 bit images down to 8 bits.\n\nFind a copy of "Color Image Quantization for Frame Buffer Display" by\nPaul Heckbert, SIGGRAPH \'82 Proceedings, page 297.  There are other\n'),
    ('How to FTP by email', ' 9) Converting between vector formats.\n    10) How to get Pixar films.\n    11) How do I draw a circle as a Bezier (or B-spline) curve?\n    12) How to order standards documents.\n    13) How to FTP by email.\n    14) How to tell whether a point is withi'),
    ('What steps should you take to obtain information about using the mail handler and software distribution?', ' exercises.  To receive information describing\nhow you can use the mail handler, simply mail graphtext@cs.brown.edu\nand put the word "Help" in the Subject line.  Use the Subject line\n"Software-Distribution" to receive information specifically concern'),
    ('How to join ACM/SIGGRAPH\n', 'trace height fields\n    24) How to find the area of a 3D polygon\n    25) How to join ACM/SIGGRAPH\n    26) Where can I find MRI and CT scan volume data?\n    27) Specific references on spatial data structures including quadtrees\n\tand octrees\n    28) Wh'),
    ('How to get general information about the\nmail server?', '/news.answers/pictures-faq/part1\nsend usenet/news.answers/pictures-faq/part2\n\nSend a message containing "help" to get general information about the\nmail server.\n\nAlso, you could check out the resources described in sections 7, 8, and\n20 above for mor'),
    ('How many tool the kit contains on image manipulation, digital halftoning?', 'rting pixels of arbitrary channels,\n    components, and bit precisions while allowing compression and machine\n    byte-order independence.  The kit contains more than 50 tools with\n    extensive support of image manipulation, digital halftoning and f'),
    ('A Fast Algorithm for Raster\nRotation', 'implementation is\nalso present in PBMPLUS.  Reference: "A Fast Algorithm for Raster\nRotation", by Alan Paeth (awpaeth@watcgl.waterloo.edu) Graphics\nInterface \'86 (Vancouver).  An article on the IM toolkit appears in\nthe same journal.  An updated vers'),
    ('What are some examples of formats that can be converted or rendered by commercial PostScript clones for PCs?', " to Sun raster format, or HPGL to\nX11 bitmap.  For example, some of the commercial PostScript clones for\nPC's allow you to render to a disk file as well as a printer.  Also,\nthe PostScript interpreters in the NeXT box and in Sun's X11/NeWs can\nbe use"),
    ('Why is assembly language used for over 100 functions in the graphical interface?', "short or floating point arithmetic to maintain the precision\n  and accuracy of the pixel format. Over 100 functions are hand-coded in\n  assembly language for maximum speed on the Intel hardware.  The entire\n  graphical interface is also written in as")
]

In [29]:
#Нужно написать эксперимент для оценки полученной коллекции
collector = ChromaCollector(splitter, embedder)
evaluator = CollectorEvaluator(collector, n_top=n_top)

stats = evaluator.explore_and_calculate(data)
print("Статистика по позициям релевантных ответов:", stats)


Search query: What are some examples of toolkits that can be used for image format conversion and basic image manipulations?,
answer: umber of toolkits for converting from one image format to
another, doing simple image manipulations such as size scaling, plus
the above-mentioned 24 -> 8, color -> gray, gray -> b&w conversions.
Here are pointers to some of them:

    xv by John Bra
Result 1: {'id': 'id_4488', 'distance': 4.962328910827637, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38403.txt'}, 'document': 'atari/Graphics/jpeg4bin.zoo.\nThese programs convert JPEG to/from PPM, GIF, Targa formats.\n\nI have not heard of any free or shareware JPEG-capable viewer for Ataris,\nbut surely there must be one by now?  Pointers appreciated.\n\nAcorn Archimedes:\n\n!Chan'}
Result 2: {'id': 'id_1696', 'distance': 5.410098552703857, 'metadata': {'source': '../../dataset/20news-bydate-train/comp.graphics/38497.txt'}, 'document': 'would like a hdftoppm type of utility, 

In [34]:
def generate(model_path, prompts, n_ctx=2000, top_k=30, top_p=0.9, temperature=0.2, repeat_penalty=1.1):
  #Реализовать генерацию текста с помощью LLM модели
  llm = Llama(model_path=model_path, n_ctx=n_ctx)

  if isinstance(prompts, str):
    prompts = [prompts]

  responses = []

  for prompt in prompts:
    response = llm(prompt,
                   top_k=top_k, top_p=top_p, temperature=temperature, 
                  #  repetition_penalty=repeat_penalty
                   )
    responses.append(response)

  return responses

In [35]:
class QuestionAndAnswers:
    # Класс для представления вопросов и ответов
    def __init__(self, question, correct_answer, generated_answer=None, prompt=None):
        self.question = question
        self.correct_answer = correct_answer
        self.generated_answer = generated_answer
        self.prompt = prompt

    def __repr__(self):
      return f"Q: {self.question}\nCorrect: {self.correct_answer}\nGenerated: {self.generated_answer}\n"

In [36]:
class Dataset:
    # Класс для представления набора данных, содержащего вопросы и ответы.
    def __init__(self, qa_list):
      self.qa_list = qa_list

def get_prompt(question, context):
  return f'Answear the question: "{question}", using context: {context}'

### Generation

In [37]:
#Нужно написать эксперимент для генерации текста (ответа на вопрос) с помощью функции generate
model_file="../../nechkasova-llm/assets/mistral-7b-openorca.Q3_K_S.gguf" 

dataset = Dataset([
    QuestionAndAnswers(
        question='What is sun?',
        correct_answer='The sun is an average star.'),
])

for qa in dataset.qa_list:
  context = example_collector.get([qa.question], n_results=1)
  qa.prompt = get_prompt(qa.question, context)

  qa.generated_answer = generate(model_file, [qa.prompt])
  print(qa)

: 

### Evaluation

In [ ]:
class BERTScoreEvaluator:
    # Класс для оценки качества сгенерированных ответов с использованием метрики BERTScore.
    def __init__(self, model_type='distilbert-base-uncased'):
        pass

    def evaluate(self, reference, generated):
        pass

    def evaluate_dataset(self, dataset: Dataset):
        pass

In [ ]:
class BLEUScoreEvaluator:
    # Класс для оценки качества сгенерированных текстов с использованием метрики BLEU.
    def __init__(self, max_order=2):
        pass

    def evaluate(self, reference, generated):
        pass

    def evaluate_dataset(self, dataset: Dataset):
        pass

In [ ]:
#Нужно написать эксперимент для оценки сгенерированых ответов